In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## The forward and backward passes

In [ ]:
#export
from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [ ]:
x_train,y_train,x_valid,y_valid = get_data()

In [ ]:
x_train.mean(),x_train.std()

(tensor(0.1304), tensor(0.3073))

In [ ]:
train_mean,train_std = x_train.mean(),x_train.std()

In [ ]:
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [ ]:
train_mean,train_std = x_train.mean(),x_train.std()

In [ ]:
x_train.mean(),x_train.std()

(tensor(3.0614e-05), tensor(1.))

In [ ]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

## Foundations version

### Basic architecture

In [ ]:
# num hidden
nh = 50

In [ ]:
# simplified kaiming init / he init
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(m)
b2 = torch.zeros(1)

In [ ]:
w1.mean(),w1.std()

(tensor(0.0001), tensor(0.0355))

In [ ]:
def relu(x): return x.clamp_min(0.)
def lin(x, w, b): return x@w + b

In [ ]:
# This should be ~ (0,1) (mean,std)...
x_valid.mean(),x_valid.std()

(tensor(-0.0058), tensor(0.9924))

In [ ]:
t = lin(x_valid, w1, b1)

In [ ]:
#...so should this
t.mean(),t.std()

(tensor(0.0726), tensor(0.9673))

In [ ]:
t = relu(lin(x_valid, w1, b1))

In [ ]:
#...actually it really should be this!
t.mean(),t.std()

(tensor(0.4210), tensor(0.5877))

From pytorch docs: `a: the negative slope of the rectifier used after this layer (0 for ReLU by default)`

$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$

In [ ]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2/m)

In [ ]:
w1.mean(),w1.std()

(tensor(0.0005), tensor(0.0506))

In [ ]:
from torch.nn import init

In [ ]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_valid, w1, b1))

In [ ]:
w1.mean(),w1.std()

(tensor(-1.3949e-05), tensor(0.0505))

In [ ]:
t.mean(),t.std()

(tensor(0.5471), tensor(0.7985))

In [ ]:
# what if...?
def relu(x): return x.clamp_min(0.) - 0.5

In [ ]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2./m )
t1 = relu(lin(x_valid, w1, b1))
t1.mean(),t1.std()

(tensor(0.0747), tensor(0.8709))

In [ ]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [ ]:
%timeit -n 10 _=model(x_valid)

8.13 ms ± 48.5 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Loss function: MSE

In [ ]:
model(x_valid).shape

torch.Size([10000, 1])

We need `squeeze()` to get rid of that trailing (,1), so we can use `mse`.

In [ ]:
def mse(output, targ): return (output.squeeze() - targ).pow(2).mean()

In [ ]:
y_train,y_valid = y_train.float(),y_valid.float()

In [ ]:
preds = model(x_train)

In [ ]:
preds.shape

torch.Size([50000, 1])

In [ ]:
mse(preds, y_train)

tensor(28.5680)

### Gradients and backward pass

In [ ]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [ ]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp>0).float() * out.g

In [ ]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [ ]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [ ]:
forward_and_backward(x_train, y_train)

## Refactor model

### Layers as classes

In [ ]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [ ]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [ ]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [ ]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [ ]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / targ.shape[0]

In [ ]:
model = Model(w1, b1, w2, b2)

In [ ]:
%time loss = model(x_train, y_train)

CPU times: user 92.3 ms, sys: 1.2 ms, total: 93.5 ms
Wall time: 46.3 ms


In [ ]:
%time model.backward()

CPU times: user 2.94 s, sys: 3.79 s, total: 6.73 s
Wall time: 3.4 s


### Module.forward()

In [ ]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [ ]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [ ]:
class Lin(Module):
    def __init__(self, ni, nf):
        self.w = torch.randn(ni,nf)/math.sqrt(ni)
        self.b = torch.zeros(nf)
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g).sum(0)
        self.b.g = out.g.sum(0)

In [ ]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [ ]:
class Model():
    def __init__(self, n_in, nh, n_out):
        self.layers = [Lin(n_in,nh), Relu(), Lin(nh,n_out)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [ ]:
model = Model(m, nh, 1)

In [ ]:
%time loss = model(x_train, y_train)

CPU times: user 94.7 ms, sys: 0 ns, total: 94.7 ms
Wall time: 45.8 ms


In [ ]:
%time model.backward()

CPU times: user 221 ms, sys: 64.6 ms, total: 286 ms
Wall time: 144 ms


### nn.Linear and nn.Module

In [ ]:
#export
from torch import nn

In [ ]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [ ]:
model = Model(m, nh, 1)

In [ ]:
%time loss = model(x_train, y_train)

CPU times: user 98.1 ms, sys: 88 µs, total: 98.2 ms
Wall time: 55.1 ms


In [ ]:
%time loss.backward()

CPU times: user 104 ms, sys: 17 ms, total: 121 ms
Wall time: 69.7 ms


## Export

In [ ]:
!./notebook2script.py 02_fully_connected.ipynb

Converted 02_fully_connected.ipynb to nb_02.py
